In [1]:
wdir='./' ## working dir
gColab=True  ## set this to False if running locally
if(gColab):
    ## Mount google drive to the machine running this nb
    from google.colab import drive
    drive.mount('/content/drive')
    ## must also specify the absolute path to the working dir
    wdir='/content/drive/MyDrive/ysNN/partC/'

Mounted at /content/drive


In [7]:
import numpy as np
from tensorflow import keras
#from tensorflow import float64 as tfloat64
import tensorflow as tf
import os,sys
sys.path.append(os.path.abspath(wdir))

#tf.keras.backend.set_floatx('float64') ## cannot do this on float32 trained NNs
nb='nb13'
wFile=wdir+nb+'_fitPolyN8'
dirValues=nb+'_dirValues.txt'
hessValues=nb+'_hessValues.txt'
gradValues=nb+'_gradValues.txt'
nnFile=nb+'_NNdata.txt'

In [3]:
#### Define model
def pwActA2(x):
    ##return x*tf.math.maximum(0.0,x)
    return x*x
def pwActS8(x):
    return x**8

nnHom=16
nun1,nun2=20,20
model=keras.Sequential()
model.add(keras.layers.Dense(nun1,input_shape=(3,),activation=pwActS8,use_bias=False))
model.add(keras.layers.Dense(nun2,activation=pwActA2,use_bias=False))
model.add(keras.layers.Dense(1,use_bias=False))

model.load_weights(wFile, skip_mismatch=False, by_name=False, options=None)

In [4]:
### Info on weights
model.summary()
ww=model.get_weights()
print(f'Number of layers = {len(ww)}')
for kk in range(len(ww)):
    print(f'shape of layer {kk}:  {ww[kk].shape}; dtype: {ww[kk].dtype}')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                60        
                                                                 
 dense_1 (Dense)             (None, 20)                400       
                                                                 
 dense_2 (Dense)             (None, 1)                 20        
                                                                 
Total params: 480 (1.88 KB)
Trainable params: 480 (1.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Number of layers = 3
shape of layer 0:  (3, 20); dtype: float32
shape of layer 1:  (20, 20); dtype: float32
shape of layer 2:  (20, 1); dtype: float32


In [5]:
### test NN-function
def nnFuncD(x,ww):
    va=pwActS8(np.dot(x,ww[0].astype(np.double)))
    va=pwActA2(np.dot(va,ww[1].astype(np.double)))
    return np.dot(va,ww[2].astype(np.double))
def nnFunc(x,ww):
    va=pwActS8(np.dot(x,ww[0]))
    va=pwActA2(np.dot(va,ww[1]))
    return np.dot(va,ww[2])

In [6]:
### Test values (directional properties)
Nphi=21
vphi=np.linspace(0,np.pi/2,Nphi);vDegs=180.0*(vphi/np.pi)
vcos=np.cos(vphi);vsin=np.sin(vphi)
vcos2,vsin2,vsc=vcos*vcos,vsin*vsin,vcos*vsin
vSnn,vRnn , vTest = np.zeros(Nphi),np.zeros(Nphi), np.zeros(Nphi)
for k in range(Nphi):
    vsx=vcos2[k]
    vsy=vsin2[k]
    vsxy=vsc[k]
    vp=np.array([vsx,vsy,vsxy]).reshape((1,3))
    #x = tf.convert_to_tensor(vp, dtype=tf.float64)
    x = tf.convert_to_tensor(vp, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x)
        yf = model(x)
    vSnn[k]=1.0/yf**(1.0/nnHom)
    ##print(tape.gradient(yf, x))
    vDX,vDY,vDXY = tape.gradient(yf, x)[0]
    vRnn[k]=(vDXY*vsc[k]-vDX*vsin2[k]-vDY*vcos2[k])/(vDX+vDY)
    vTest[k]=1.0/(nnFunc(vp,ww))**(1.0/nnHom)
print('row,      sigma,            test_Sigma,          r-Value')
with open(wdir+dirValues,'w') as ff:
    for kk in range(Nphi):
        ff.write(f'{vSnn[kk]}, {vRnn[kk]}\n')
        print(kk, ', ', vSnn[kk],', ', vTest[kk],',', vRnn[kk])

row,      sigma,            test_Sigma,          r-Value
0 ,  1.0004215240478516 ,  1.0004215937070144 , 0.496087908744812
1 ,  0.9922065138816833 ,  0.9922065088316397 , 0.4550919234752655
2 ,  0.9752882122993469 ,  0.9752881702414287 , 0.3974383771419525
3 ,  0.9563760757446289 ,  0.9563760157592777 , 0.35747215151786804
4 ,  0.9395022392272949 ,  0.9395022512724366 , 0.33675071597099304
5 ,  0.9262538552284241 ,  0.9262539155967862 , 0.3261992335319519
6 ,  0.9169812798500061 ,  0.9169813640958888 , 0.3167116343975067
7 ,  0.9114213585853577 ,  0.9114212981532052 , 0.30360138416290283
8 ,  0.9088269472122192 ,  0.908826927571801 , 0.28831079602241516
9 ,  0.9079607725143433 ,  0.9079608084501629 , 0.27635079622268677
10 ,  0.9072330594062805 ,  0.9072330077744664 , 0.2730328440666199
11 ,  0.9052842259407043 ,  0.9052842027163606 , 0.2798762023448944
12 ,  0.9019768834114075 ,  0.9019768667914619 , 0.2933731973171234
13 ,  0.8988373279571533 ,  0.8988373270978545 , 0.307929307222366

In [18]:
### Raw (yield function) Gradient and Hessian test values

#### sigma = [x1,x2,x3] = [vcos2,vsin2,vsc]
x1=tf.Variable(vcos2[:],name='x1',trainable=False)
x2=tf.Variable(vsin2[:],name='x2',trainable=False)
x3=tf.Variable(vsc[:],name='x3',trainable=False)
g1,g2,g3=[],[],[]
with tf.GradientTape(persistent=True) as tape2:
    tape2.watch([x1,x2,x3])
    with tf.GradientTape() as tape1:
        tape1.watch([x1,x2,x3])
        vy=(model(tf.stack([x1,x2,x3],axis=1)))**(1/nnHom)
    g1,g2,g3=tape1.gradient(vy,[x1,x2,x3])

h11=tape2.gradient(g1,x1,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
h12=tape2.gradient(g1,x2,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
h13=tape2.gradient(g1,x3,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
h22=tape2.gradient(g2,x2,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
h23=tape2.gradient(g2,x3,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
h33=tape2.gradient(g3,x3,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
print('row,     H11,                 H12,                   H13,                   H22,                    H23,                     H33')
with open(wdir+hessValues,'w') as ff:
    for kk in range(Nphi):
        ff.write(f'{h11[kk]}, {h12[kk]}, {h13[kk]}, {h22[kk]}, {h23[kk]}, {h33[kk]}\n')
        print(kk,',',h11[kk],',',h12[kk],',',h13[kk],',',h22[kk],',',h23[kk],',',h33[kk])

row,     H11,                 H12,                   H13,                   H22,                    H23,                     H33
0 , 1.91513620961814e-07 , -1.785598868764282e-08 , 9.312560678154114e-08 , 2.1379594802856445 , 0.08881819248199463 , 5.037006378173828
1 , 0.025027871131896973 , -0.05060603469610214 , -0.31405770778656006 , 1.9948818683624268 , 0.48601579666137695 , 3.9522218704223633
2 , 0.06979179382324219 , -0.11270010471343994 , -0.4227980077266693 , 1.6939043998718262 , 0.4432716369628906 , 2.5992326736450195
3 , 0.11574134230613708 , -0.14057183265686035 , -0.4483455419540405 , 1.3696216344833374 , 0.2567063570022583 , 1.8058669567108154
4 , 0.1728469580411911 , -0.15783508121967316 , -0.4806826114654541 , 1.1297225952148438 , 0.11869660019874573 , 1.4408233165740967
5 , 0.24604998528957367 , -0.18907570838928223 , -0.5156983137130737 , 0.9905465245246887 , 0.04617135226726532 , 1.2258816957473755
6 , 0.3317691385746002 , -0.2439972460269928 , -0.5268082618713379 , 0

In [21]:
print('row,      Grad_1,               Grad_2,               Grad_3')
vg1,vg2,vg3=g1.numpy(),g2.numpy(),g3.numpy()
ff=open(wdir+gradValues,'w')
for kk in range(Nphi):
    ff.write(f'{vg1[kk], {vg2[kk]}, vg3[kk]}\n')
    print(kk,', ', vg1[kk], ', ', vg2[kk], ', ', vg3[kk])
ff.close()

row,      Grad_1,               Grad_2,               Grad_3
0 ,  0.9995786547660828 ,  -0.33145034313201904 ,  0.02070513367652893
1 ,  0.9857540726661682 ,  -0.29311418533325195 ,  0.38320431113243103
2 ,  0.954120397567749 ,  -0.2203935980796814 ,  0.6469549536705017
3 ,  0.9157493114471436 ,  -0.14548403024673462 ,  0.8268811702728271
4 ,  0.8727517127990723 ,  -0.07649756222963333 ,  0.9605100154876709
5 ,  0.8235530257225037 ,  -0.00948403961956501 ,  1.0693155527114868
6 ,  0.7667922377586365 ,  0.06143353506922722 ,  1.1597331762313843
7 ,  0.701179027557373 ,  0.1404796540737152 ,  1.2324981689453125
8 ,  0.6242101192474365 ,  0.2298693060874939 ,  1.2877278327941895
9 ,  0.5322717428207397 ,  0.3306329846382141 ,  1.3245983123779297
10 ,  0.4237975776195526 ,  0.4420502185821533 ,  1.3386573791503906
11 ,  0.30513855814933777 ,  0.5579335689544678 ,  1.3229202032089233
12 ,  0.1916978806257248 ,  0.6654993295669556 ,  1.2762037515640259
13 ,  0.09744629263877869 ,  0.75317174

In [22]:
### Export the NN details and weights (column-wise)
vdeg=[8,2]   ####    homogeneity degrees of the activation functions
nLay=len(ww)  ### number of layers
ff=open(wdir+nnFile,'w')
ff.write(f'{nLay}\n')  ###   the (total) number of layers
for kk in range(nLay-1): ### only the hidden layers
    ff.write(f'{ww[kk].shape[1]}\n') ###  number of units in layer
    ff.write(f'{vdeg[kk]}\n')  ####   homogeneity degree
for kk in range (nLay):
    for jj in range(ww[kk].shape[1]):
        for ii in range(ww[kk].shape[0]):
            ff.write(f'{ww[kk][ii,jj]}\n')   #### weight
ff.close()

In [23]:
!jupyter nbconvert --to html  {wdir+nb+'_ExportWeights.ipynb'}

[NbConvertApp] Converting notebook /content/drive/MyDrive/ysNN/partC/nb13_ExportWeights.ipynb to html
[NbConvertApp] Writing 624931 bytes to /content/drive/MyDrive/ysNN/partC/nb13_ExportWeights.html
